# Goals: Automated grade the summaries based on rubrics
## Data and codebook:
1.   Summary Score.csv
2.   Summary.csv
3.   Summary Score Data Codebook
4.   Meta data: Rubric, source text, example summary response
### Located in https://drive.google.com/file/d/16JMuxnxDFts41AkqfvMpeiOdLI9j1nHT/view?usp=sharing

1. Data preprocessing

In [8]:
#Load packages
#!pip install openai
#!pip install openai chromadb
#!pip install tiktoken
#!pip install pathlib
#!pip install transformers

In [9]:
# !pip uninstall -y httpx
# !pip install httpx==0.28.1

In [10]:
import os,openai, signal, subprocess, tiktoken
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from google.colab import files, drive, userdata # This is how we access Key

In [11]:
# Mount Google Drive
drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/EDUC 6192/LLM_2025_Hands_on/Hands-on_data'
os.chdir(base_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# read files and remove sentence transformation, which has already been automated identified in the ITS
def read_file(file_name, sheet_name):
    df = pd.read_excel(file_name, sheet_name = sheet_name)

    columns_to_drop = [col for col in df.columns
                   if any(substring.lower() in str(col).lower()
                       for substring in ['SentenceTransformation', 'axis=1'])]

    df = df.drop(columns=columns_to_drop)

     # Modify columns in place
    df.columns = df.columns.str.replace(r'\s+', '', regex=True)

    print(df.head(2))
    print(df.columns)
    return df

df = read_file('1st_Human_Grading.xlsx', sheet_name = "Original")


   SummaryID  AssignedID  RealLesson   Structure  \
0          1  Formal5_02           2   Causation   
1          2  Formal5_02           3  Comparison   

                                             Summary TopicSentence_dc  \
0  Floods are one of the most common natural disa...             High   
1  The two most destructive hurricanes to ever hi...             High   

  Content_dc Mechanics_dc SignalWord_dc TopicSentence_K  ... Mechanics_K  \
0     Medium         High          High            High  ...      Medium   
1       High         High        Medium            High  ...         Low   

  SignalWord_K TopicSentence_mlg Content_mlg Mechanics_mlg SignalWord_mlg  \
0          Low            Medium        High        Medium            Low   
1       Medium            Medium      Medium        Medium            Low   

  TopicSentence_BW Content_Bw Mechanics_Bw SignalWord_BW  
0           Medium       High         High          High  
1             High     Medium       Medium  

In [13]:
df.shape

(32, 21)

2. Automated grading

In [14]:
# Securely fetch the API key from Secrets
api_key = userdata.get('LAAI_OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

2.1 Prompt

In [15]:
# ── Your writing instruction ────────────────────────────────
INSTRUCTION = """
Please read the texts carefully and take your time to write a summary with 50-100 words with your own words.\
You should include a topic sentence to state the main idea, followed by the important supporting information. \
Remember to use the signal words to explicitly express your ideas.
"""

In [16]:
# ── Source text ────────────────────────────────
SOURCE_TEXT= {
     "texts": [
        {
            "RealLesson": 5,
            "text": """Kobe Bryant and Michael Jordan are two of the greatest shooting guards in NBA history. Although they played 8 games against one another, they were part of different eras, with Jordan’s career spanning 1984 – 2003 (with several periods of retirement between 1993 and 2003) and Bryant’s career beginning in 1996.
Kobe Bryant joined the NBA straight after graduating from high school and has played for the Los Angeles Lakers for his entire career. He was the 13th draft pick by the Charlotte Hornets in 1996, and was then immediately traded to the Lakers. By the end of his first season, he averaged 15.5 points per game and was winner of the 1997 Slam Dunk Contest. Since 1999, he has been one of the league’s premier shooting guards and has won five NBA championships.
Michael Jordan left the University of North Carolina to enter the NBA draft in 1984. He was the Chicago Bulls’ 3rd overall pick. In his first season, he averaged 28.2 points per game and appeared on the cover of Sports Illustrated just one month into his professional career. He was voted an All-Star starter in his rookie season, and then Rookie of the Year. He broke his foot in his second season and missed 64 games, but recovered in time for the playoffs. He retired in 1993 after the murder of his father and signed a minor league baseball contract with the Chicago White Sox. He returned to the NBA in 1995, and retired again in 1999, before becoming part owner and president of the Washington Wizards. In 2001, he returned to the NBA again as a player for the Washington Wizards, with the intention of donating his salary to the September 11 relief efforts. He played his final season in 2002-2003."""
        },
        {
            "RealLesson": 6,
            "text": """Adding aerobic exercise to your routine is an excellent way to build your cardiovascular fitness. Running and walking are both excellent aerobic exercises. Both will help promote weight loss, improve your sleep, elevate your mood, boost your energy level, decrease blood pressure and cholesterol levels, and decrease the risk of cancer, diabetes, and heart disease.
Walking and running are low-cost, easy-to-do anywhere, year-round activities. Both are social activities - you can walk or run or with a friend. Because running is more rigorous than walking, I think that you should select a running program to maximize aerobic conditioning in the least time possible. Those who do not wish to run can obtain the same health and fitness benefits by walking. Walking is good exercise for those who are just starting to workout, or for those with health problems. For the significantly overweight, walking can be less stressful on the body. Those who desire can slowly build up to running.
In order to get any benefit from a workout, it has to be one that you enjoy and will do day after day. So if you prefer walking, then do it! As a walker, you might have to walk a little more or eat a little less to achieve your weight goal versus running. Walkers can make up that difference by going farther in distance. Distance wise, a 160 pound person burns about 100 calories per mile walking or running. If you look at it on a calories-burned-per-hour basis, a person will burn more calories by running an hour rather than walking an hour.
Another important Formality to consider when examining the difference between running and walking is that because of the repetitive nature of running, the risk of injury is greater. Running is considered high impact exercise. This can cause injury to the hip, knee and ankle joints. However, walking is a low impact activity and is potentially less damaging to the joints.
If you choose to run, reduce your risk of injury by running on the best surfaces in this order: Grass, woodland trails, earth, cinders and man-made tracks. Wear good quality shoes that have been fitted for you by a shoe expert. Run with correct form and technique, gradually increase the mileage that you run and get some guidance from a personal trainer."""
        },
        {
            "RealLesson": 7,
            "text": """There are several benefits to exercising that most people are unaware of. Although exercising is time consuming, it can affect your health in a positive way. Exercising can control weight by preventing excess weight gain or aid in weight loss. It does this by burning calories; the more intense the workout the more calories you burn. Exercise can also help prevent health problems such as heart disease, diabetes, depression, and types of cancers, such as colon and breast cancer.
Exercising can improve your overall mood by stimulating your brain to make you feel happier and more relaxed. Exercising increases muscle and reduces fat in your body. Exercising also gives you more energy. Do not forget that exercising will not be effective if you do not have a proper diet. The foods you consume are just as important as the exercises you perform. Having a bad diet will cause your exercising to be unproductive and you may not see any positive changes in your body. Hence, exercising combined with a proper diet will allow you to see positive changes in your body that you may have never thought possible. So, get out there and go exercise!"""
        },
        {
            "RealLesson": 8,
            "text": """Many people have heard of diabetes but very few understand where it may come from or what may cause it. Having diabetes simply means that someone has high levels of sugar in their blood and it usually is a lifelong disease. Insulin is a hormone produced in the body that controls blood sugar. This type of diabetes can be caused by too little insulin or resistance to insulin. When food is consumed and broken down in the body it is used for energy, this then changes to sugar. With diabetes your body cannot move sugar into muscle, fat or liver cells which is caused by two things. Either the pancreas, the organ that makes insulin, does not make enough insulin or your cells do not respond to insulin normally.
There are several symptoms that occur with diabetes such as, fatigue, blurry vision, weight loss, and excess thirst. If you or anyone you may know has any of these symptoms, contact a doctor as soon as possible. There are two types of diabetes: Type 1 can happen at any age, it develops over a slow period of time, and there is no prevention for it and Type 2 normally happens in adulthood, is caused by high rates of obesity, and can be prevented with a healthy, active lifestyle that includes diet and exercise. Diabetes can cause harmful health complications over the years such as kidney problems, heart attacks, eye problems, and body sores."""
        },
        {
            "RealLesson": 10,
            "text": """Butterfly and moth have some things in common, such as flying large-winged insects with two long antennae and a four-stage life cycle called metamorphosis. Butterflies and moths also have other things in common. Both are in the group of insects called lepidoptera, and have three main body parts—a head, thorax, and abdomen. Both have a long tongue called a proboscis, which uncoils for feeding, working kind of like a drinking straw. Both have tiny colored scales covering their wings. When touched the scales come off and look like dust on your fingers.
Butterfly and moth have a number of differences as well. Most butterflies are diurnal meaning they are active during the day, whereas most moths are nocturnal meaning they are active at night. The bodies of butterflies are slender and smoother than the thicker and hairier moths. Butterfly antennae have thick knobs on their ends, while moth antennae are feathery or plain and with no knobs. Most butterflies rest with their wings folded together above their bodies; however, most moths rest with their wings spread out to the sides. Most butterflies have colorful wings, but many moths aren’t so bright and colorful, though there are some that are. While both insects go through a complete metamorphosis cycle—egg, larva (caterpillar), pupa, adult—their pupa stages are slightly different. A butterfly caterpillar forms a chrysalis, and hangs from a tree branch or other support. A moth caterpillar usually spins a silk cocoon, which might then be hidden among leaves or other debris on the ground."""
        },
        {
            "RealLesson": 3,
            "text": """Two of the most destructive hurricanes to hit the United States in recent years were Hurricane Andrew in 1992 and Hurricane Katrina in 2005. Both hurricanes inflicted massive destruction and damage to parts of the southeast United States.
When Hurricane Andrew made landfall in Homestead, Florida, it was a category 5 hurricane—the strongest measure for hurricanes. At landfall its winds measured 175 miles per hour and it dumped seven inches of rain on southern Florida. With storm surges nearly 17 feet, the damage was extensive. Estimated property damage was $25 billion and 26 people died as a result of the hurricane.
Hurricane Katrina struck the Mississippi and Louisiana coast as a category 3 hurricane with winds of 120 miles per hour at landfall. Although Katrina was not as strong as Andrew, it was far more destructive and deadly. This is because the city of New Orleans was in its path. A storm surge of 12 feet reached the city and caused the protective levees to break, flooding 80% of the city. More than 1,800 people died and property damage was about $81 billion.
Both hurricanes caused suffering and devastation to the people who lived through them. In measuring hurricane strength, Andrew was the stronger of the two. However, Katrina inflicted more destruction and fatalities because its path was through New Orleans."""
        },
        {
            "RealLesson": 2,
            "text": """Floods are second only to fire as the most common of all natural disasters. They occur almost everywhere in the world, resulting in widespread damage and even death. Consequently, scientists have long tried to perfect their ability to predict floods. So far, the best that scientists can do is to recognize the potential for flooding in certain conditions. There are a number of conditions, from deep snow on the ground to human error, that cause flooding.
When deep snow melts it creates a large amount of water. Although deep snow alone rarely causes floods, when it occurs together with heavy rain and sudden warmer weather it can lead to serious flooding. If there is a fast snow melt on top of frozen or very wet ground, flooding is more likely to occur than when the ground is not frozen. Melting snow also contributes to high water levels in rivers and streams. Whenever rivers are already at their full capacity of water, heavy rains will result in the rivers overflowing and flooding the surrounding land.
Although scientists cannot always predict exactly when floods will occur, they do know a great deal about when floods are likely, or probably, going to occur. Deep snow, ice-covered rivers, and weak dams are all strong conditions for potential flooding. Hopefully, this knowledge of why floods happen can help us reduce the damage they cause."""
        },
        {
            "RealLesson": 9,
            "text": """The deterioration of the middle of the labor market is easy to misinterpret, because of its multiple roots. During the 1970s, entry into the work force of an unusual number of people resulted in too many workers for the jobs available and decreased wages. This decline is associated with the explosive growth in world trade since 1960.
As manufacturing technologies become more mobile and multinational firms freer, production jobs have moved from the U.S. to countries where wages are low. In addition, technology has caused shifts in the job market. For example, less American workers were required to make steel in 1980 than in 1960 because of new machines. The high rate of unemployment caused by these developments has decreased wages, since it forces unskilled workers to compete for jobs with unemployed people who will do the work for less.
Although these Formalitys have had an effect on the job market, middle-level jobs disappear as a result of the ways technological gains are being distributed. Both the firm and consumers as a group benefit when a machine replaces a production worker. The loss falls on the worker who is displaced. If that loss is generalized to millions of high-paid workers, those workers suffer and the economy as a whole suffers a loss of power. Thus the lack of a way to distribute some of the financial gains from technology to the work force comes back to haunt the entire economy."""
        }
    ]
}


In [17]:
# ── Your rubric text ────────────────────────────────
RUBRIC_TEXT = """
Integer (2, 1, 0)

Topic Sentence
- 2: The summary begins with a clear topic sentence that states the main idea.
- 1: The summary has a topic sentence that touches upon the main idea.
- 0: The summary does not state the main idea.

Content Inclusion & Exclusion
- 2: Major details are stated economically and arranged in a logical order. No minor or unimportant details or reflections are added.
- 1: Some but not all major details are stated and not necessarily in a logical order. Some minor or unimportant details or reflections are added.
- 0: Few major details are stated and not necessarily in a logical order. Many minor or unimportant details or reflections are added.

Mechanics and Grammar
- 2: There are few or no errors in mechanics, usage, grammar or spelling.
- 1: There are some errors in mechanics, usage, grammar or spelling that to some extent interfere with meaning.
- 0: There are serious errors in mechanics, usage, grammar or spelling that make the summary difficult to understand.

Signal Words
- 2: Uses clear and accurate signal words to connect information.
- 1: Uses several clear and accurate signal words to connect information, with occasional imprecision.
- 0: Uses few or inaccurate/ill-fitting signal words; connections are unclear.
""".strip()

In [18]:
# Your three examples (Good/Bad/Medium)
EXAMPLES = """
EXAMPLE 1 (Good):
"The deterioration of the middle of the labor market can be misinterpreted due to its several roots. First, too many people enter into the work force caused a decrease in wages due to the high competition for the jobs available. Second, manufacturing technologies caused production jobs to move from the U.S. to low-wage countries, which caused the high rate of unemployment. Finally, technological gains are being distributed from production workers to machines, which cause the disappearance of middle-level jobs."
Expected grading (JSON sketch):
{
  "criteria": {
    "Topic Sentence": 2,
    "Content Inclusion & Exclusion": 2,
    "Mechanics and Grammar": 2,
    "Signal Words": 2
  },
  "rationale": "Clear main idea up front; major points covered and organized; minimal errors; effective connectors (e.g., 'First', 'due to', 'cause")."
}

EXAMPLE 2 (Medium):
"Butterflies look like moths in many ways. Butterflies have slender and smoother bodies and their antennae have thick knobs on their ends. Butterflies have colorful wings, and their wings fold together above their bodies when they rest. But butterflies are different from moths. A butterfly’s chrysalis hangs from a tree branch or other support, whereas a moth’s cocoon is hidden among leaves or debris. Most butterflies are active during the day."
Expected grading (JSON sketch):
{
  "criteria": {
    "Topic Sentence": 2,
    "Content Inclusion and Exclusion": 1,
    "Mechanics Grammar": 1,
    "Signal Words": 1
  },
  "rationale": "Clear main idea; covers many key points but includes minor/awkward bits; some grammar issues; transitions present but not always precise."
}

EXAMPLE 3 (Bad):
"Hurricane Andrew ranked at a 5 on the category measure based on how severe the hurricane was. This hurricane was so severe it caused $25 billion of damage. Hurricane Katrina is at a 3 on the category measure. The higher the category measure is, the more severe destruction the hurricane causes. So, Hurricane Andrew caused more damage and death than Hurricane Katrina."
Expected grading (JSON sketch):
{
  "criteria": {
    "Topic Sentence": 0,
    "Content Inclusion & Exclusion": 1,
    "Mechanics Grammar": 1,
    "Signal Words": 1
  },
  "rationale": "Opens with reluctance rather than main idea; key ideas incomplete/disordered; frequent grammar issues; connectors used but imprecise and not clarifying."
}
""".strip()


In [19]:
content = f"""
You are an expert of summary writing assessor. Grade a student summary against a fixed rubric with strict, evidence-based consistency. Use ONLY the SOURCE_TEXT provided (no outside knowledge). If the summary mentions facts not supported by SOURCE_TEXT, treat them as inaccuracies per the rubric.

**SOURCE_TEXT**
{SOURCE_TEXT}

**TASK INSTRUCTIONS**
{INSTRUCTION}

**RUBRIC (authoritative; do not invent rules)**
{RUBRIC_TEXT}

** CALIBRATION EXAMPLES (for qualitative feel only)**
{EXAMPLES}

**SCORING & OUTPUT INSTRUCTIONS:**
1) Evidence-only: Judge solely by alignment with SOURCE_TEXT and the rubric. Ignore prior beliefs or world knowledge.
2) Faithfulness: Penalize fabricated, contradicted, or irrelevant content according to the rubric.
3) Objectivity: Do not infer intent. Evaluate what is written, not what you think the student meant.
4) Determinism: When in doubt, choose the lowest score that is fully justified by the rubric definitions.
5) Signal words: If the assignment implies a structure (e.g., Comparison or Causation), assess transition words accordingly (e.g., Comparison: both, however, whereas, while, in contrast, similarly, different(ly); Causation: because, therefore, as a result, leads to, due to, thus, consequently). Use the rubric’s exact criteria to score.
6) Mechanics: Judge grammar, usage, punctuation, capitalization, and basic readability per rubric. Minor typos may reduce the score only if they impede clarity per rubric guidance.
7) Output Format: You MUST output your final assessment using the following exact structure and labels. Do not add any other text, headers, or commentary before or after this block.

### Topic Sentence Score: [0/1/2]
### Content Inclusion & Exclusion Score: [0/1/2]
### Mechanics and Grammar Score: [0/1/2]
### Signal Words Score: [0/1/2]
### Rational_TS: [Your concise 1-2 sentence reasoning for the Topic Sentence score here.]
### Rational_CIE: [Your concise 1-2 sentence reasoning for the Content Inclusion & Exclusion score here.]
### Rational_MG: [Your concise 1-2 sentence reasoning for the Mechanics and Grammar score here.]
### Rational_SW: [Your concise 1-2 sentence reasoning for the Signal Words score here.]

**BEGIN ASSESSMENT NOW:**

""".strip()

2.2 GPT-generated response

In [20]:
def process_dataframe(df):
    responses = []
    for index, row in df.iterrows():
        prompt = str(row["Summary"])

        chat_completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
         #   max_tokens=2000,
            messages=[{"role": "system", "content": content + '\n'},
             {"role": "user", "content": prompt},])
        result = chat_completion.choices[0].message.content
        responses.append(result)
        print(result)
        print("____________")

    return responses


In [21]:
responses = process_dataframe(df)


### Topic Sentence Score: 2
### Content Inclusion & Exclusion Score: 1
### Mechanics and Grammar Score: 2
### Signal Words Score: 1
### Rational_TS: The summary begins with a clear topic sentence that states the main idea about floods being common natural disasters.
### Rational_CIE: While it includes some major details about factors causing floods, it lacks a logical order and omits important details from the source text, such as the conditions that increase the likelihood of flooding.
### Rational_MG: The mechanics and grammar are mostly correct, with no significant errors that impede understanding.
### Rational_SW: The summary uses some signal words, but they are not effectively connecting the information, leading to a lack of clarity in the relationships between ideas.
____________
### Topic Sentence Score: 2
### Content Inclusion & Exclusion Score: 1
### Mechanics and Grammar Score: 1
### Signal Words Score: 1
### Rational_TS: The summary begins with a clear topic sentence that st

In [22]:
for i, response in enumerate(responses):
    df.at[i, "response"] = response

df

,SummaryID,AssignedID,RealLesson,Structure,Summary,TopicSentence_dc,Content_dc,Mechanics_dc,SignalWord_dc,TopicSentence_K,...,SignalWord_K,TopicSentence_mlg,Content_mlg,Mechanics_mlg,SignalWord_mlg,TopicSentence_BW,Content_Bw,Mechanics_Bw,SignalWord_BW,response
0,1,Formal5_02,2,Causation,Floods are one of the most common natural disa...,High,Medium,High,High,High,...,Low,Medium,High,Medium,Low,Medium,High,High,High,### Topic Sentence Score: 2\n### Content Inclu...
1,2,Formal5_02,3,Comparison,The two most destructive hurricanes to ever hi...,High,High,High,Medium,High,...,Medium,Medium,Medium,Medium,Low,High,Medium,Medium,Low,### Topic Sentence Score: 2\n### Content Inclu...
2,3,Formal5_02,5,Comparison,Michael Jordan and Kobe Bryant are two of the ...,High,Medium,High,Medium,High,...,Low,Medium,Low,Medium,Medium,High,Medium,High,Medium,### Topic Sentence Score: 1\n### Content Inclu...
3,4,Formal5_02,6,Comparison,Walking and running are both very good exercis...,Medium,Medium,High,High,High,...,High,Medium,High,High,Medium,High,Medium,High,High,### Topic Sentence Score: 1\n### Content Inclu...
4,5,Formal5_02,7,Causation,Exercising has a tremendous amount of benefits...,High,Medium,High,Medium,High,...,High,High,High,High,Medium,High,High,High,High,### Topic Sentence Score: 2\n### Content Inclu...
5,6,Formal5_02,8,Causation,Even though many people have heard of diabetes...,High,High,Medium,High,High,...,Low,Medium,Medium,Medium,Low,High,High,High,High,### Topic Sentence Score: 2\n### Content Inclu...
6,7,Formal5_02,9,Causation,The United States job market has deteriorated....,Medium,Medium,High,Low,Medium,...,Medium,Medium,Medium,Medium,Low,High,Medium,High,Low,### Topic Sentence Score: 1\n### Content Inclu...
7,8,Formal5_02,10,Comparison,The butterfly and the moth have a lot of thing...,Medium,Medium,High,Medium,Medium,...,Medium,Medium,Medium,High,Low,High,High,Medium,High,### Topic Sentence Score: 2\n### Content Inclu...
8,9,Formal9_02,2,Causation,flood is one of the disaster factor after fire...,Low,Medium,Low,Medium,Low,...,Low,Low,Low,Low,Low,Low,Medium,Low,High,### Topic Sentence Score: 1\n### Content Inclu...
9,10,Formal9_02,3,Comparison,united states have been effected by 2 hurrycan...,Low,Medium,Low,Medium,Low,...,Low,Low,Low,Low,Medium,High,High,Low,Low,### Topic Sentence Score: 1\n### Content Inclu...


In [23]:
print(df['response'])

0     ### Topic Sentence Score: 2\n### Content Inclu...
1     ### Topic Sentence Score: 2\n### Content Inclu...
2     ### Topic Sentence Score: 1\n### Content Inclu...
3     ### Topic Sentence Score: 1\n### Content Inclu...
4     ### Topic Sentence Score: 2\n### Content Inclu...
5     ### Topic Sentence Score: 2\n### Content Inclu...
6     ### Topic Sentence Score: 1\n### Content Inclu...
7     ### Topic Sentence Score: 2\n### Content Inclu...
8     ### Topic Sentence Score: 1\n### Content Inclu...
9     ### Topic Sentence Score: 1\n### Content Inclu...
10    ### Topic Sentence Score: 0\n### Content Inclu...
11    ### Topic Sentence Score: 1\n### Content Inclu...
12    ### Topic Sentence Score: 1\n### Content Inclu...
13    ### Topic Sentence Score: 1\n### Content Inclu...
14    ### Topic Sentence Score: 1\n### Content Inclu...
15    ### Topic Sentence Score: 1\n### Content Inclu...
16    ### Topic Sentence Score: 2\n### Content Inclu...
17    ### Topic Sentence Score: 2\n### Content I

In [24]:
df['TopicSentence_GPT'] = df['response'].str.extract(r'### Topic Sentence Score:\s*(\d+)').astype(int)
df['Content_GPT'] = df['response'].str.extract(r'### Content Inclusion & Exclusion Score:\s*([\d.]+)').astype(int)
df['Mechanics_GPT'] = df['response'].str.extract(r'### Mechanics and Grammar Score:\s*(\d+)').astype(int)
df['SignalWords_GPT'] = df['response'].str.extract(r'### Signal Words Score:\s*([\d.]+)').astype(int)
df['Score_GPT'] = df['TopicSentence_GPT'] + df['Content_GPT'] + df['Mechanics_GPT'] + df['SignalWords_GPT']
df['Score_GPT'] = df['Score_GPT'].astype(int)
df['TS_Rational'] = df['response'].str.extract(r'### Rational_TS:\s*(.+)')
df['CIE_Rational'] = df['response'].str.extract(r'### Rational_CIE:\s*(.+)')
df['MG_Rational'] = df['response'].str.extract(r'### Rational_MG:\s*(.+)')
df['SW_Rational'] = df['response'].str.extract(r'### Rational_SW:\s*(.+)')
print(df)

    SummaryID   AssignedID  RealLesson   Structure  \
0           1   Formal5_02           2   Causation   
1           2   Formal5_02           3  Comparison   
2           3   Formal5_02           5  Comparison   
3           4   Formal5_02           6  Comparison   
4           5   Formal5_02           7   Causation   
5           6   Formal5_02           8   Causation   
6           7   Formal5_02           9   Causation   
7           8   Formal5_02          10  Comparison   
8           9   Formal9_02           2   Causation   
9          10   Formal9_02           3  Comparison   
10         11   Formal9_02           5  Comparison   
11         12   Formal9_02           6  Comparison   
12         13   Formal9_02           7   Causation   
13         14   Formal9_02           8   Causation   
14         15   Formal9_02           9   Causation   
15         16   Formal9_02          10  Comparison   
16         17  Formal15_03           2   Causation   
17         18  Formal15_03  

In [25]:
df.shape

(32, 31)

In [26]:
df.to_excel('GPT_evaluation_1st.xlsx', index=False)
files.download('GPT_evaluation_1st.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>